In [1]:
import os
os.chdir("../..")
import pandas as pd

from scripts.python.ts_utils import *
from scripts.python.utils import *
from scripts.python.PdfParse import *

In [3]:
solomon_folder = os.getcwd() + "/data/tourism/solomon/"
solomon_pdfs = [solomon_folder + file
                for file in os.listdir(solomon_folder) if ".pdf" in file]

## Official Statistics
### Parsing

In [4]:
solomon_2020s = [pdf for pdf in solomon_pdfs if "2020" in pdf]

In [5]:
# Read the pdf file
df = load_pdf(filepath=solomon_2020s[1],
              search_string="Table 3",
              table_page=8)
df = df.iloc[:13, ].dropna(how="all", axis=1).dropna(thresh=3, axis=0)
df.head(5)

,NaN,NaN,NaN,NaN,NaN,NaN,NUMBER,NaN,NaN,NaN,NaN,NaN
0,January,"1,602","1,235","1,600","1,383","1,590","1,415 1,259","1,655","1,514","2,081","1,750","1,857"
1,February,"1,422","1,209","1,658","1,591","1,544","1,523 1,465","1,707","1,635","1,855","1,891","1,471"
2,March,"1,249","1,766","1,861","1,677","2,184","1,816 1,675","1,675","1,732","2,360","2,378",752
3,April,"1,499","1,820","1,736","1,839","2,021","1,514 1,750","1,799","2,013","2,250","2,106",2
4,May,"1,393","1,137","1,760","1,968","1,857","1,462 1,681","1,896","1,851","2,003","2,434",23


In [6]:
yr_range = len(df.columns) - 1 + 1  # drop month column and include splitted
yr_lst = [i for i in range(2020 - yr_range + 1, 2020 + 1)]

for colname in df.columns:
    if type(colname) == str and len(colname) > 4 and colname != "Month":
        break

col_idx = df.columns.to_list().index(colname)

In [7]:
## Expand the column and keep the sequence by
## creating before/splitted/after dataframe
df_precol, df_postcol = df.iloc[:, :col_idx],  df.iloc[:, col_idx+1:]

precol_lst = df_precol.columns.to_list()
precol_lst[0], precol_lst[1:] = "Month", yr_lst[:col_idx-1]
df_precol.columns = precol_lst
df_postcol.columns = yr_lst[-len(df_postcol.columns):]

splitted = df.iloc[:, col_idx].str.split(" ", expand=True)
splitted.columns = yr_lst[col_idx-1: -col_idx+1]

In [10]:
temp_df = pd.concat([df_precol, splitted, df_postcol], axis=1)
temp_df = remove_separator(temp_df)


for col in temp_df.columns[1:]:
    temp_df[col] = temp_df[col].astype(int)

temp_df.head(5)

,Month,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,January,1602,1235,1600,1383,1590,1415,1259,1655,1514,2081,1750,1857
1,February,1422,1209,1658,1591,1544,1523,1465,1707,1635,1855,1891,1471
2,March,1249,1766,1861,1677,2184,1816,1675,1675,1732,2360,2378,752
3,April,1499,1820,1736,1839,2021,1514,1750,1799,2013,2250,2106,2
4,May,1393,1137,1760,1968,1857,1462,1681,1896,1851,2003,2434,23


In [11]:
temp_df_tr = temp_df.T
temp_df_tr.columns = temp_df_tr.iloc[0].to_list()
temp_df_tr = (temp_df_tr
              .drop(index="Month")
              .reset_index()
              .rename({"index": "Year"}, axis=1))

temp_df_tr.to_csv(
    solomon_folder + "intermediate/visitor_by_month_2009-20.csv", encoding="utf-8")

In [20]:
temp_df

,Month,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,January,1602,1235,1600,1383,1590,1415,1259,1655,1514,2081,1750,1857
1,February,1422,1209,1658,1591,1544,1523,1465,1707,1635,1855,1891,1471
2,March,1249,1766,1861,1677,2184,1816,1675,1675,1732,2360,2378,752
3,April,1499,1820,1736,1839,2021,1514,1750,1799,2013,2250,2106,2
4,May,1393,1137,1760,1968,1857,1462,1681,1896,1851,2003,2434,23
5,June,1724,1797,2009,3122,2317,1949,2131,1984,2561,2768,2523,24
6,July,1603,2264,2185,2865,2574,2114,2077,2319,2295,2862,2537,52
7,August,1520,1845,1900,2133,2022,1639,1426,1918,2589,2313,2484,39
8,September,1659,2153,2159,2309,2291,1730,2211,1428,2397,2663,2695,103
9,October,1709,1813,1931,1929,2057,1798,1748,2257,2500,2366,2761,60


In [12]:
sb = (temp_df_tr.melt(id_vars="Year")
      .rename({"variable": "month", "value": "number"}, axis=1))
get_adf_df(sb, ["number"])

,Test Statistic,p-value,# Lags Used,Number of Observations Used,Critical Value (1%),Critical Value (5%),Critical Value (10%)
number,-5.753993,5.877363e-07,12.0,143.0,-3.476927,-2.881973,-2.577665


In [24]:
df_19_20 = temp_df.iloc[:-1, -2:].unstack().reset_index().rename({0: "Total"}, axis=1)
df_19_20.head(5)

,level_0,level_1,Total
0,2019,0,1750
1,2019,1,1891
2,2019,2,2378
3,2019,3,2106
4,2019,4,2434


In [13]:
print(check_quality(temp_df_tr, ["Year"], "Total"))

[6]


## Aviation Statistics

In [21]:
aviation_path = os.getcwd() + "/data/tourism/aviation_seats_flights_pic.xlsx"
aviation = pd.read_excel(aviation_path)
aviation.head(5)

,Country,ISO,Region,Date,Aircraft_type,Seats_arrivals_domestic,Seats_arrivals_interregional,Seats_arrivals_intraregional,Seats_arrivals_intl,Seats_arrivals_total,Available_seat_kilometers,Number_of_flights_domestic,Number_of_flights_interregional,Number_of_flights_intraregional,Number_of_flights_intl,Number_of_flights_total
0,Fiji,FJ,East Asia & Pacific,2019-01-01,passenger,839,273,3480,3753,4592,1.430416e+07,8,1,10,11,19
1,Fiji,FJ,East Asia & Pacific,2019-01-02,passenger,974,313,3471,3784,4758,1.495610e+07,8,1,10,11,19
2,Fiji,FJ,East Asia & Pacific,2019-01-03,passenger,1190,443,3675,4118,5308,1.592143e+07,10,2,12,14,24
3,Fiji,FJ,East Asia & Pacific,2019-01-04,passenger,831,586,3159,3745,4576,1.457334e+07,7,2,12,14,21
4,Fiji,FJ,East Asia & Pacific,2019-01-05,passenger,744,273,4752,5025,5769,1.773449e+07,7,1,12,13,20


In [22]:
sb_avi = (aviation[(aviation.ISO == "SB") & (aviation.Aircraft_type == "passenger")]
          .reset_index()
          .drop("index", axis=1)
          [["Date", "Number_of_flights_intl", "Number_of_flights_total",
            "Seats_arrivals_intl", "Seats_arrivals_total"]])

dates = pd.DataFrame(pd.date_range(start="2019-01-01",
                                   end="2020-12-31"), columns=["Date"])

sb_avi = dates.merge(sb_avi, how="left", on="Date")
sb_avi["Date"] = pd.to_datetime(sb_avi["Date"])
sb_avi["Month"], sb_avi["Year"] = sb_avi["Date"].dt.month, sb_avi["Date"].dt.year
sb_avi_19_20 = sb_avi.groupby(by=["Year", "Month"]).sum().reset_index()
sb_avi_19_20.head(5)

,Year,Month,Number_of_flights_intl,Number_of_flights_total,Seats_arrivals_intl,Seats_arrivals_total
0,2019,1,69.0,69.0,10036.0,10036.0
1,2019,2,66.0,66.0,9471.0,9471.0
2,2019,3,69.0,71.0,9693.0,9965.0
3,2019,4,66.0,71.0,9532.0,10212.0
4,2019,5,69.0,73.0,9859.0,10403.0


In [25]:
sb_merged = pd.concat([sb_avi_19_20, df_19_20], axis=1).drop(["level_0", "level_1"], axis=1)
sb_merged = remove_separator(sb_merged)
sb_merged["Total"] = sb_merged["Total"].astype(float)

In [26]:
sb_merged[["Total", "Seats_arrivals_intl"]].describe()

,Total,Seats_arrivals_intl
count,24.000000,24.000000
mean,1389.458333,6357.375000
std,1160.284784,4291.692772
min,2.000000,510.000000
25%,49.250000,1239.000000
50%,1803.500000,9278.000000
75%,2446.500000,9734.500000
max,3053.000000,11255.000000


In [27]:
print(sb_merged["Total"]/sb_merged["Seats_arrivals_intl"])

0     0.174372
1     0.199662
2     0.245332
3     0.220940
4     0.246881
5     0.264771
6     0.266464
7     0.266010
8     0.292363
9     0.274072
10    0.203909
11    0.284344
12    0.166607
13    0.232680
14    0.110556
15    0.003922
16    0.033824
17    0.029126
18    0.042484
19    0.047794
20    0.060875
21    0.073529
22    0.012862
23    0.018320
dtype: float64


## EDA

### Correlation and Cross-correlation

In [28]:
from scipy.stats import pearsonr

corr_seats, _ = pearsonr(sb_merged["Seats_arrivals_total"], sb_merged["Total"])
corr_seats_intl, _ = pearsonr(
    sb_merged["Seats_arrivals_intl"], sb_merged["Total"])
print(f"Pearson Correlation between FlightRadar's Seats Arrival and VU's census data is{corr_seats: .4f}.\n",
      f"Pearson Correlation between FlightRadar's # of International Seats Arrival and VU's census data is{corr_seats_intl: .4f}.")

Pearson Correlation between FlightRadar's Seats Arrival and VU's census data is 0.9521.
 Pearson Correlation between FlightRadar's # of International Seats Arrival and VU's census data is 0.9503.


In [29]:
sb_cc = cross_corr_df(sb_merged, "Seats_arrivals_intl", "Total")
sb_cc.head(5)

,lag,cross_corr_coef
0,0,0.950256
1,1,0.915791
2,2,0.791296
3,3,0.683567
4,4,0.496011


### Stationarity

In [30]:
incl_cols = ["Total", "Seats_arrivals_intl"]
get_adf_df(sb_merged, incl_cols)

,Test Statistic,p-value,# Lags Used,Number of Observations Used,Critical Value (1%),Critical Value (5%),Critical Value (10%)
Total,-0.451626,0.901116,0.0,23.0,-3.752928,-2.998500,-2.638967
Seats_arrivals_intl,-0.559959,0.879775,9.0,14.0,-4.012034,-3.104184,-2.690987


In [31]:
# Difference once to check stationarity
sb_diff = sb_merged.diff().dropna()
sb_diff2 = sb_diff.diff().dropna()
get_adf_df(sb_diff2, incl_cols)

,Test Statistic,p-value,# Lags Used,Number of Observations Used,Critical Value (1%),Critical Value (5%),Critical Value (10%)
Total,-10.592363,6.478634e-19,0.0,21.0,-3.788386,-3.013098,-2.646397
Seats_arrivals_intl,-12.078784,2.257563e-22,0.0,21.0,-3.788386,-3.013098,-2.646397


Both `Total` and `Seats_arrivals_intl` are somewhat stationary after differencing once.

### Granger Causality

In [33]:
grangers_causation_matrix(sb_diff, incl_cols, maxlag=6)

,Total_x,Seats_arrivals_intl_x
Total_y,1.0,0.0067
Seats_arrivals_intl_y,0.0,1.0000
